In [ ]:
from math import exp
from random import seed
import random
from random import randrange
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
import time

''' Initialize the network '''
def init_network(numinputs, numhiddenlayer, numoutputs):
    net=list()
    # hidden layer
    HiddenLayer = [{'weights':[np.random.uniform() for i in range(numinputs+1)]} for i in range(numhiddenlayer)]
    net.append(HiddenLayer)
    OutputLayer = [{'weights':[np.random.uniform() for i in range(numhiddenlayer+1)]} for i in range(numoutputs)]
    net.append(OutputLayer)
    return net

''' Transfer activation function for the neuron '''

def transferFunction(actn):
    return 1.0/(1.0+exp(-actn))

'''Calculation of activation of neuron from an input '''

def activateFunction(wts, inp):
    activaN = wts[-1]
    for i in range(len(wts)-1):
        activaN+=wts[i]*inp[i]
    return activaN

''' Forward Propagation of the network '''

def forwardPropagationNet(netK, row):
    inps = row
    for layer in netK:
        newinps=[]
        for neuron in layer:
            activaN = activateFunction(neuron['weights'], inps)
            neuron['output'] = transferFunction(activaN)
            newinps.append(neuron['output'])
        inps = newinps
    return inps, neuron['weights']


''' Derivative of the Transfer Function '''
def derivativeTransferFunction(op):
    return (op)*(1-op)

''' Backward Propagation of the network '''

def backwardPropagationNet(netK, observed):
    for i in reversed(range(len(netK))):
        layer = netK[i]
        errorlst = list()
        if i != len(netK)-1:
            for j in range(len(layer)):
                e = 0.0
                for neuron in netK[i+1]:
                    e += (neuron['weights'][j] * neuron['delta'])
                errorlst.append(e)
        else:
            for k in range(len(layer)):
                neuron = layer[k]
                errorlst.append(observed[k] - neuron['output'])
        for p in range(len(layer)):
            neuron = layer[p]
            neuron['delta'] = errorlst[p] * derivativeTransferFunction(neuron['output'])


''' Update Weights of the Network '''
def updateWeightsFunction(netK, row, learningRate):
    for i in range(len(netK)):
        inp = row[:-1]
        if i != 0:
            inp = [neuron['output'] for neuron in netK[i-1]]
        for neuron in netK[i]:
            for j in range(len(inp)):
                neuron['weights'][j] += learningRate * neuron['delta'] * inp[j]
            neuron['weights'][-1] += learningRate * neuron['delta']


''' Training the Network.'''

def trainingNetwork(netK, trainData, learningRate, epochs, numoutputs):
    #print("Learning Rate of the network = %.3f" % (learningRate))
    #epocherror = 0
    reg_lambda = 0.0001
    for epoch in range(epochs):
        errorsum = 0
        for row in trainData:
            outputs, weights = forwardPropagationNet(netK,row)
            observed = [0 for i in range(numoutputs)]
            #print len(observed)
            #print observed
            #print row[-1]
            #raise KeyboardInterrupt
            observed[int(row[-1])] = 1
            errorsum -= sum([(observed[i]-outputs[i])**2 for i in range(len(observed))])
            errorsum += (reg_lambda/float(2))*np.sum(np.square(weights))
            backwardPropagationNet(netK,observed)
            updateWeightsFunction(netK,row,learningRate)
        #print('Number_of_Epoch=%d, trainingError=%.3f' % (epoch,errorsum))
        #epocherror=errorsum
    #print("Training Accuracy of the Model after Training is %.3f" % (100+epocherror))


''' Cross Validation function '''
def splitByCrossValidation(data,nFolds):
    dataSplit = list()
    copyData = data
    foldNumber = int(len(data) / nFolds)
    for i in range(nFolds):
        fold = list()
        while len(fold) < foldNumber :
            idx = randrange(len(copyData))
            fold.append(copyData.pop(idx))
        dataSplit.append(fold)
    return dataSplit

''' Prediction Function '''
def predict(netK,row):
    out,_ = forwardPropagationNet(netK,row)
    #print "out", out
    return out.index(max(out))

''' Accuracy Metric '''

def accuracyCalculation(A,P):
    hit = 0
    for i in range(len(P)):
        if A[i] == P[i]:
            hit+=1
    return (hit/float(len(A))) * 100.0

def printPR(tp,tn,fn,fp):
    precision=0; recall=0
    #print tp, tn, fn, fp
    try:
        precision = tp/float(tp+fp)
    except ZeroDivisionError:
        print #"error"
    try:
        recall = tp/float(tp+fn)
    except ZeroDivisionError:
        print #"error"
    return precision, recall, fp/float(fp+tn)
    
''' this dataset involves two class problem. So we take 1 as Positive and 0 as Negative'''
''' Not a good programming style ********* Mind it '''
def confusionmatrix(A,P):
    tp=0; fp=0; tn=0; fn=0;
    for i in range(len(A)):
        if int(A[i]) == P[i]:
            if int(A[i]) == 1:
                tp+=1
            else:
                tn+=1
        else:
            if int(A[i]) == 0:
                fp+=1
            else:
                fn+=1
    p1,r1,fpr1 = printPR(tp,tn,fn,fp) 
    print p1,r1,fpr1
    tp=0; fp=0; tn=0; fn=0;
    
def confusionmatrix_new(A,P,label):
    tp=0; fp=0; tn=0; fn=0;
    for i in range(len(A)):
        if int(A[i]) == P[i]:
            if int(A[i]) == label:
                tp+=1
            else:
                tn+=1
        else:
            if int(A[i]) == label:
                fn+=1
            if P[i] == label:
                fp+=1
    return tp, tn, fp, fn
    '''
    for i in range(len(A)):
        if int(A[i]) == P[i]:
            if int(A[i]) == 0:
                tp+=1
            else:
                tn+=1
        else:
            if int(A[i]) == 1:
                fp+=1
            else:
                fn+=1
    p0,r0,fpr0 = printPR(tp,tn,fn,fp)
    print (p0+p1)/float(2), (r0+r1)/float(2), (fpr0+fpr1)/float(2)
    '''
   #printPR(tp,tn,fn,fp)
    
    #fscore = 2*precision*recall/float(precision+recall)
    #return precision, recall

def normlize_dataframe(df):
    '''

    :param df:
    :return: normalized dataframe after Min-max feature scaling (normalization)
    '''
    df_norm = (df - df.min()) / (df.max()-df.min())
    return df_norm


''' Main Function '''

if __name__ == '__main__':
    seed(1)
    df = pd.read_csv('../datasets/fluML.csv')
    df = df.dropna()
    ''' Shifting the target variable to the end'''
    Flu = df.Flu.copy(deep=True)
    df = df.drop('Flu',1)
    df['Flu'] = Flu
    #print df.head()
    #raise KeyboardInterrupt
    newdf = df.copy(deep=True)
    cols_list = df.columns.values
    #newdf = newdf.drop(cols_list[0], 1)
    newdf = newdf.iloc[:,9:17]
    newdf = newdf.sample(frac=1).reset_index(drop=True)
    newdf = normlize_dataframe(newdf)
    newdf = shuffle(newdf)
    #print newdf.shape
    #print df.shape
    #raise KeyboardInterrupt
    #print('Data Loaded and Prepared.')
    dataset=list()
    for rows in newdf.values:
        dataset.append(list(rows))
    #print np.asarray(dataset).shape
    #raise KeyboardInterrupt
    numFolds = 7
    #print('Now we are performing CrossValidation')
    dataSplit = splitByCrossValidation(dataset,numFolds)
    resultscore = list()
    trainset=list()
    testset=list()
    tpr_list = list(); fpr_list=list()
    TP0 = 0; TN0 = 0; FP0=0; FN0=0;
    TP1 = 0; TN1 = 0; FP1=0; FN1=0;
    
    for fld in dataSplit:
        trainset = list(dataSplit)
        trainset.remove(fld)
        #raise KeyboardInterrupt
        #print trainset.remove(fld)
        trainset = sum(trainset,[])
        testset =list()
        for r in fld:
            copyrow = list(r)
            testset.append(copyrow)
            copyrow[-1] = None
        numinputs = len(trainset[0])-1
        #print("Number of the features of the dataset %d" %(numinputs))
        numoutputs = len(set(row[-1] for row in trainset))
        #print("Number of the labels of the dataset %d" %(numoutputs))
        #print("Initializing the Network .......")
        ''' Passing the number of features, number of hidden neurons and number of outputs'''
        netK = init_network(numinputs,3,numoutputs)
        #print("Network Initialized. Training of the Network, begins ....")
        ''' Passing data, network, learning rate, epochs and number of outputs'''
        trainingNetwork(netK, trainset, 0.01, 600, numoutputs)
        #print('Printing the output of each layer of the network')
        #for layer in netK:
         #   print(layer)
        predictions = list()
        for row in testset:
            pred = predict(netK, row)
            predictions.append(pred)
        actualval = [row[-1] for row in fld]
        #print actualval
        #print "\n"
        #print predictions
        #raise KeyboardInterrupt
        result= accuracyCalculation(actualval,predictions)
        tp0, tn0, fp0, fn0 = confusionmatrix_new(actualval, predictions, 0)
        TP0+=tp0; TN0+=tn0; FP0+=fp0; FN0+=fn0;
        tp1, tn1, fp1, fn1 = confusionmatrix_new(actualval, predictions, 1)
        TP1+=tp1; TN1+=tn1; FP1+=fp1; FN1+=fn1;
        resultscore.append(result)
    print "for label 0:", TP0, TN0, FP0, FN0
    print "for label 1:", TP1, TN1, FP1, FN1
    print "True Positive rate (recall/TPR) : ", float(TP0+TP1)/float(TP0+TP1+FN0+FN1)
    print "False Positive rate (FPR):", float(FP0+FP1)/float(FP0+FP1+TN0+TN1)
    print("Accuracy of the Model is %.3f" %(sum(resultscore)/float(len(resultscore))))
    print('Jobs Ends!!')